In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


## Download en & ja dataset

In [2]:
# target text
text_en = """This paper describes SentencePiece, a
language-independent subword tokenizer
and detokenizer designed for Neural-based
text processing, including Neural Machine
Translation.""".replace("\n", " ")

text_ja = "この文書では、ニューラル機械翻訳を含むニューラルベースのテキスト処理用に設計された、言語に依存しないサブワードトークナイザーおよびデトークナイザーである SentencePiece について説明します。"
text_ko = "이 문서에서는 신경망 기계 번역을 포함하여 신경망 기반 텍스트 처리용으로 설계된 언어 독립적 하위 단어 토크나이저 및 디토큰나이저인 SentencePiece에 대해 설명합니다."

In [3]:
from tqdm import tqdm
from datasets import load_dataset

dataset = load_dataset("opus100", "en-ja")

with open("opus100-en.txt", "w") as f_en, open("opus100-ja.txt", "w") as f_ja:
    for it in dataset["train"]:
        f_en.write(it["translation"]["en"] + "\n")
        f_ja.write(it["translation"]["ja"] + "\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Usage in transformers

## WordPiece on English

In [4]:
# https://huggingface.co/learn/nlp-course/chapter6/8?fw=pt#building-a-wordpiece-tokenizer-from-scratch
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    Regex,
)
import time
import os
from copy import deepcopy

In [5]:
_text = deepcopy(text_en)
print(_text)

#normalizers.Sequence([normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()])
normalizer = normalizers.BertNormalizer()

_text = normalizer.normalize_str(_text)
print(_text)


pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(),
                                         pre_tokenizers.Digits(individual_digits=True)])
_text = pre_tokenizer.pre_tokenize_str(_text)
print(_text)


post_processor = processors.TemplateProcessing(
    single="[CLS] $A [EOS]",
    special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
)

def build_wordpiece_tokenizer():
    tokenizer = Tokenizer(model = models.WordPiece(unk_token="[UNK]"))
    tokenizer.normalizer = normalizers.Sequence([normalizers.NFD(),
                                                 normalizers.Lowercase(),
                                                 normalizers.StripAccents()])
    tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(),
                                                       pre_tokenizers.Digits(individual_digits=True)])
    tokenizer.post_processor = processors.TemplateProcessing(
        single="[CLS] $A [EOS]",
        special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
    )
    return tokenizer
tokenizer = build_wordpiece_tokenizer()

This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.
this paper describes sentencepiece, a language-independent subword tokenizer and detokenizer designed for neural-based text processing, including neural machine translation.
[('this', (0, 4)), ('paper', (5, 10)), ('describes', (11, 20)), ('sentencepiece', (21, 34)), (',', (34, 35)), ('a', (36, 37)), ('language', (38, 46)), ('-', (46, 47)), ('independent', (47, 58)), ('subword', (59, 66)), ('tokenizer', (67, 76)), ('and', (77, 80)), ('detokenizer', (81, 92)), ('designed', (93, 101)), ('for', (102, 105)), ('neural', (106, 112)), ('-', (112, 113)), ('based', (113, 118)), ('text', (119, 123)), ('processing', (124, 134)), (',', (134, 135)), ('including', (136, 145)), ('neural', (146, 152)), ('machine', (153, 160)), ('translation', (161, 172)), ('.', (172, 173))]


In [6]:
tokenizer.get_vocab()

{}

In [7]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

tokenizer.train(files=["opus100-en.txt"], trainer=trainer)

os.makedirs("./tokenizers", exist_ok=True)
tokenizer.save("./tokenizers/en-wordpiece.json")

In [8]:
for i, (key, value) in enumerate(tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)


needed 2064
##of 2902
hates 7000
management 7267
jail 3544
abe 7826
stood 4627
##い 323
##rick 8099
##gues 7461
に 191
##umn 3232
injury 8224
##ne 1271
##ched 6738
sque 6538
##ped 1463
hay 4419
##pp 526
kn 465


In [9]:
encoded = tokenizer.encode(text_en)
print("len:", len(encoded.tokens), encoded.tokens)

print(tokenizer.decode(encoded.ids))
print("------------------")

print(tokenizer.normalizer.normalize_str(text_en))
print(tokenizer.decode(encoded.ids).replace(" ##", ""))

len: 40 ['[CLS]', 'this', 'paper', 'describe', '##s', 'sentence', '##pi', '##ec', '##e', ',', 'a', 'language', '-', 'independent', 'sub', '##word', 'to', '##ken', '##izer', 'and', 'det', '##oken', '##izer', 'designed', 'for', 'neur', '##al', '-', 'based', 'text', 'process', '##ing', ',', 'including', 'neur', '##al', 'machine', 'translation', '.', '[EOS]']
this paper describe ##s sentence ##pi ##ec ##e , a language - independent sub ##word to ##ken ##izer and det ##oken ##izer designed for neur ##al - based text process ##ing , including neur ##al machine translation .
------------------
this paper describes sentencepiece, a language-independent subword tokenizer and detokenizer designed for neural-based text processing, including neural machine translation.
this paper describes sentencepiece , a language - independent subword tokenizer and detokenizer designed for neural - based text processing , including neural machine translation .


In [10]:
encoded = tokenizer.encode(text_ko)
print("len:", len(encoded.tokens), encoded.tokens)

encoded = tokenizer.encode(text_ja)
print("len:", len(encoded.tokens), encoded.tokens)

len: 24 ['[CLS]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '.', '[EOS]']
len: 13 ['[CLS]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'sentence', '##pi', '##ec', '##e', '[UNK]', '[UNK]', '[EOS]']


In [11]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from transformers import BertTokenizerFast
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

text_en = "hello-world!"
encoded = bert_tokenizer.encode(text_en)
print("len:", len(encoded))

print(text_en)
print(bert_tokenizer.decode(encoded))
# note again `language-independent` -> `language - independent`

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


len: 6
hello-world!
[CLS] hello - world! [SEP]


## Sentencepiece on English

In [12]:
_text = deepcopy(text_en)
print(_text)

normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "), # replace 2+ spaces with one.
    ]
)


_text = normalizer.normalize_str(_text)
print(_text)


pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Metaspace(),
                                         pre_tokenizers.Digits(individual_digits=True)])
_text = pre_tokenizer.pre_tokenize_str(_text)
print(_text) # unlike `Whitespace`, `Metaspace` preserves space with '_'.


post_processor = processors.TemplateProcessing(
    single="[CLS] $A [EOS]",
    special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
)

def build_sentencepiece_tokenizer():
    tokenizer = Tokenizer(model = models.WordPiece(unk_token="[UNK]"))
    tokenizer.normalizer = normalizers.Sequence(
        [
            normalizers.Replace("``", '"'),
            normalizers.Replace("''", '"'),
            normalizers.NFKD(),
            normalizers.StripAccents(),
            normalizers.Replace(Regex(" {2,}"), " "), # replace 2+ spaces with one.
        ]
    )
    tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Metaspace(),
                                                       pre_tokenizers.Digits(individual_digits=True)])
    tokenizer.post_processor = processors.TemplateProcessing(
        single="[CLS] $A [EOS]",
        special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
    )
    return tokenizer
tokenizer = build_sentencepiece_tokenizer()

hello-world!
hello-world!
[('▁hello-world!', (0, 12))]


In [13]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

tokenizer.train(files=["opus100-en.txt"], trainer=trainer)

os.makedirs("./tokenizers", exist_ok=True)
tokenizer.save("./tokenizers/en-sentpiece.json")

In [14]:
for i, (key, value) in enumerate(tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)


▁but 712
▁sing 2198
##house 5080
##eless 6458
##D 299
##iring 6573
##anna 1415
##ces.
 7657
▁baby, 6100
▁then?
 4097
▁folder 6969
▁corner 7006
##ale.
 7584
▁met.
 9135
##gged 5346
##Κ 385
##:
 1174
##ly?
 3753
##ah.
 4289
##idered 5707


In [15]:
encoded = tokenizer.encode(text_en)
print("len:", len(encoded.tokens), encoded.tokens)

print(tokenizer.decode(encoded.ids))
print("------------------")

print(tokenizer.normalizer.normalize_str(text_en))
print(tokenizer.decode(encoded.ids).replace("##", "").replace(" ", "")[1:].replace('▁', ' '))

len: 8 ['[CLS]', '▁hell', '##o-', '##wo', '##r', '##ld', '##!', '[EOS]']
▁hell ##o- ##wo ##r ##ld ##!
------------------
hello-world!
hello-world!


In [16]:
from tokenizers import Tokenizer
from transformers import XLNetTokenizer
xlnet_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

encoded = xlnet_tokenizer.encode(text_en)
print("len:", len(encoded))

print(text_en)
print(xlnet_tokenizer.decode(encoded))


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

len: 6
hello-world!
hello-world!<sep><cls>


In [17]:
import sentencepiece as spm
t1 = time.time()
_ = spm.SentencePieceTrainer.train(
    input='opus100-en.txt',
    model_prefix='en-sp',
    model_type="bpe",
    vocab_size=10000,
    )
print("en-sentpiece google time:", time.time() - t1)


en-sentpiece google time: 27.119542121887207


In [18]:
sp = spm.SentencePieceProcessor(model_file='./en-sp.model')

encoded = sp.encode(text_en)
print("len:", len(encoded), encoded)
print("------------------")
print(text_en)
print(sp.decode(encoded))

len: 5 [4002, 9948, 5058, 76, 9954]
------------------
hello-world!
hello-world!


In [19]:
wp_tokenizer = build_wordpiece_tokenizer()
sp_tokenizer = build_sentencepiece_tokenizer()

special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

t1 = time.time()
wp_tokenizer.train(files=["opus100-en.txt"], trainer=trainer)
print("en-wordpiece time:", time.time() - t1)

t1 = time.time()
sp_tokenizer.train(files=["opus100-en.txt"], trainer=trainer)
print("en-sentpiece time:", time.time() - t1)

os.makedirs("./tokenizers", exist_ok=True)
wp_tokenizer.save("./tokenizers/en-wordpiece.json")
sp_tokenizer.save("./tokenizers/en-sentpiece.json")

en-wordpiece time: 21.138954401016235
en-sentpiece time: 46.73186945915222


## WordPiece & sentencepiece on Japanese

In [20]:
wp_tokenizer = build_wordpiece_tokenizer()
sp_tokenizer = build_sentencepiece_tokenizer()

special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

import time
t1 = time.time()
wp_tokenizer.train(files=["opus100-ja.txt"], trainer=trainer)
print("ja-wordpiece time:", time.time() - t1)

t1 = time.time()
sp_tokenizer.train(files=["opus100-ja.txt"], trainer=trainer)
print("ja-sentpiece time:", time.time() - t1)

os.makedirs("./tokenizers", exist_ok=True)
wp_tokenizer.save("./tokenizers/ja-wordpiece.json")
sp_tokenizer.save("./tokenizers/ja-sentpiece.json")

ja-wordpiece time: 92.02121114730835
ja-sentpiece time: 149.39432191848755


In [21]:
encoded = wp_tokenizer.encode(text_ja)
print("len:", len(encoded.tokens), encoded.tokens)

print(wp_tokenizer.decode(encoded.ids))
print("------------------")

encoded = sp_tokenizer.encode(text_ja)
print("len:", len(encoded.tokens), encoded.tokens)
print(sp_tokenizer.decode(encoded.ids))

len: 73 ['[CLS]', 'この', '##文', '##書', '##ては', '、', 'ニュー', '##ラ', '##ル', '##機', '##械', '##翻', '##訳', '##を', '##含', '##む', '##ニ', '##ュー', '##ラ', '##ル', '##ヘース', '##のテ', '##キ', '##スト', '##処', '##理', '##用', '##に', '##設', '##計', '##された', '、', '言', '##語', '##に', '##依', '##存', '##しない', '##サ', '##フ', '##ワー', '##ト', '##ト', '##ーク', '##ナ', '##イ', '##サー', '##お', '##よ', '##ひ', '##テ', '##ト', '##ーク', '##ナ', '##イ', '##サー', '##てある', 'se', '##nt', '##en', '##c', '##e', '##p', '##i', '##e', '##c', '##e', 'に', '##ついて', '##説明', '##します', '。', '[EOS]']
この ##文 ##書 ##ては 、 ニュー ##ラ ##ル ##機 ##械 ##翻 ##訳 ##を ##含 ##む ##ニ ##ュー ##ラ ##ル ##ヘース ##のテ ##キ ##スト ##処 ##理 ##用 ##に ##設 ##計 ##された 、 言 ##語 ##に ##依 ##存 ##しない ##サ ##フ ##ワー ##ト ##ト ##ーク ##ナ ##イ ##サー ##お ##よ ##ひ ##テ ##ト ##ーク ##ナ ##イ ##サー ##てある se ##nt ##en ##c ##e ##p ##i ##e ##c ##e に ##ついて ##説明 ##します 。
------------------
len: 76 ['[CLS]', '▁この', '##文', '##書', '##ては', '##、', '##ニ', '##ュー', '##ラ', '##ル', '##機', '##械', '##翻', '##訳', '##を', '##含', '##む', '##ニ', '##ュー', '#

In [22]:
print("- Wordpiece --------")
for i, (key, value) in enumerate(wp_tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)

print("- Sentencepiece --------")
for i, (key, value) in enumerate(sp_tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)


- Wordpiece --------
##茜 5788
##乏 5776
催 515
ツ 289
党 551
癌 2303
円 568
フ 299
##？ 4169
果 1747
##藻 6383
##の子 8135
##を飲 9602
##靡 7626
##膀 6489
##餞 6500
酢 3359
##針 5472
##琥 6880
淳 2007
- Sentencepiece --------
蛾 2938
##渾 7106
宴 1087
##記 4493
##嚼 6344
##娠 5804
##かを 9852
##螺 5591
索 2573
笛 2494
鵞 3704
##暖 5288
##んてす
 8847
##希 5224
##紀 5391
腱 2747
##旅 4044
鑑 3436
##蜥 7552
##變 7642
